In [ ]:
import torch
import torchvision.transforms as T
from PIL import Image
import numpy as np
import os, random

# preprocessing
transform = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor()
])

to_pil = T.ToPILImage()

# ---- LSB embedding / extraction ----
def embed_key_lsb(image: Image.Image, key: int):
    """Embed 8-bit key into the least significant bit of the image pixels"""
    img_arr = np.array(image).copy()
    flat = img_arr.flatten()
    for i in range(8):  # store 8 bits
        flat[i] = (flat[i] & ~1) | ((key >> i) & 1)
    return Image.fromarray(flat.reshape(img_arr.shape))

def extract_key_lsb(image: Image.Image):
    """Extract 8-bit key from LSB"""
    img_arr = np.array(image)
    flat = img_arr.flatten()
    key = 0
    for i in range(8):
        key |= (flat[i] & 1) << i
    return key

# ---- simple XOR encryption ----
def encrypt_half(tensor, key):
    return torch.clamp(tensor + (key/255.0), 0, 1)

def decrypt_half(tensor, key):
    return torch.clamp(tensor - (key/255.0), 0, 1)


In [ ]:
import gradio as gr
import torch
import numpy as np
from torchvision import transforms
from PIL import Image

# ========== UTILS ==========
to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()

def preprocess(img: Image.Image) -> torch.Tensor:
    img = img.resize((256, 256))
    return to_tensor(img).unsqueeze(0)  # [1,3,256,256]

def postprocess(tensor: torch.Tensor) -> Image.Image:
    tensor = tensor.detach().cpu().squeeze(0).clamp(0,1)
    return to_pil(tensor)

# ========== ENCODER / DECODER FUNCTIONS ==========
def perform_encoding(encoder, cover1, cover2, secret):
    secret_top = secret[:, :, 0:128, :]
    secret_bottom = secret[:, :, 128:256, :]
    stego1 = encoder(cover1, secret_top)
    stego2 = encoder(cover2, secret_bottom)
    return stego1, stego2

def perform_decoding(decoder, stego1, stego2):
    secret_top_recon = decoder(stego1)
    secret_bottom_recon = decoder(stego2)
    secret_recon = torch.cat([secret_top_recon, secret_bottom_recon], dim=2)
    return secret_recon
